# Web Scrapping in Python


**TODO pour moi avant le TME**

1.   Faire la version étudiant en enlevant les réponses. Ne garder qu'un code générique pour la carte qu'ils devront adapter



The first objective of this notebook is to discover the `request` and `BeautifulSoup` libraries to crawl a table on a Wikitable page, build a dataframe, and create a map.

*   `request` and [urllib](https://docs.python.org/3/library/urllib.html#module-urllib) for requestion REST API
*   [BeautifulSoup4](https://www.crummy.com/software/BeautifulSoup/bs4/doc/) to inspect webpages

Please note that this part serves as an initiation of web scrapping but you will need to learn by yourself to make the project. It is inspired from notebooks published by Galiana, Lino. 2023. Python Pour La Data Science. https://doi.org/10.5281/zenodo.8229676. Exercices target different sources of information, the code needs to be adapted.











In [17]:
!pip install -q lxml

from bs4 import BeautifulSoup
import lxml
import pandas
import urllib
import pandas as pd

from urllib import request

## Exercice 1 : Scrap a table on a wikipedia page

We would like to display on a map the location of Summer Olympic Games since 1896. We will use a [Wikipedia page](https://fr.wikipedia.org/wiki/Jeux_olympiques) to scrap the associated table.
Below is the code to extract the content of the page using `request` and display its title using `BeautifulSoup`.

In [18]:
jo = "https://fr.wikipedia.org/wiki/Jeux_olympiques"

request_text = request.urlopen(jo).read()
print(request_text[:1000])
page = BeautifulSoup(request_text, "lxml")
print(page.find("title"))

b'<!DOCTYPE html>\n<html class="client-nojs vector-feature-language-in-header-enabled vector-feature-language-in-main-page-header-disabled vector-feature-page-tools-pinned-disabled vector-feature-toc-pinned-clientpref-1 vector-feature-main-menu-pinned-disabled vector-feature-limited-width-clientpref-1 vector-feature-limited-width-content-enabled vector-feature-custom-font-size-clientpref-1 vector-feature-appearance-pinned-clientpref-1 vector-feature-night-mode-enabled skin-theme-clientpref-day vector-sticky-header-enabled vector-toc-available" lang="fr" dir="ltr">\n<head>\n<meta charset="UTF-8">\n<title>Jeux olympiques \xe2\x80\x94 Wikip\xc3\xa9dia</title>\n<script>(function(){var className="client-js vector-feature-language-in-header-enabled vector-feature-language-in-main-page-header-disabled vector-feature-page-tools-pinned-disabled vector-feature-toc-pinned-clientpref-1 vector-feature-main-menu-pinned-disabled vector-feature-limited-width-clientpref-1 vector-feature-limited-width-c

Our objective here is to extract the different information in the first table "Jeux olympiques d'été" and to build a data frame.

To proceed, you will have to follow these steps:
*   Find the table
*   Collect each line of the table
*   Retrieve the different columns and transform them into text format. Also, use `strip` to format the value into a proper text format (e.g., without useless spaces). Store these lines (formated as a table of columns) in a table.

**Warning: We have two location for 1940. You have therefore to manage this case: identify the attribute indicating this fact and write a code to collect those lines and insert in the previous table in the right place**

* Collect headers of the HTML table
* Build a data frame from the result table and the headers








In [54]:
#[student]
tables = page.find_all("table", {"class": "wikitable"})

summer_olympics_table = tables[0]

headers = [header.text.strip() for header in summer_olympics_table.find_all("th")]
rows = []
for row in table.find_all("tr")[1:]:
    cols = row.find_all(["th", "td"])
    cols_text = [col.get_text(strip=True) for col in cols]

    if "1940" in cols_text[0]:
        # Create separate entries with the correct structure
        base_cols = cols_text[:2]  # Year and Olympiad number
        rows.append(base_cols + ["Tokyo", "Japon", "Asie"])
        rows.append(base_cols + ["Helsinki", "Finlande", "Europe"])
    else:
        rows.append(cols_text)

df = pd.DataFrame(rows, columns=headers)

df = df[df['Année'] != 'Helsinki']

print(df)

   Année Olympiade(Nº de l’olympiade[NB 1])         Ville hôte  \
0   1896                                  I         Athènes(1)   
1   1900                                 II           Paris(1)   
2   1904                                III     Saint-Louis(1)   
3   1906             •(Intercalaires[NB 2])         Athènes(2)   
4   1908                                 IV         Londres(1)   
5   1912                                  V       Stockholm(1)   
6   1916                  VI(annulés[NB 3])             Berlin   
7   1920                                VII          Anvers(1)   
8   1924                               VIII           Paris(2)   
9   1928                                 IX       Amsterdam(1)   
10  1932                                  X     Los Angeles(1)   
11  1936                                 XI          Berlin(1)   
12  1940                 XII(annulés[NB 4])              Tokyo   
13  1940                 XII(annulés[NB 4])           Helsinki   
15  1944  

## Exercice 2: Locate places on a map

The objective here is to identify organizer cities on a map.
You will have to code the following steps:
* Collect the URL of each city
* Go the this page using `urllib.request`
* Find the coordinates
* Store cities and coordinates in a data frame

In [88]:
import pandas as pd
from urllib.request import urlopen
from bs4 import BeautifulSoup
import re
import time

# Read the content of the page
request_text = urlopen(jo).read()
page = BeautifulSoup(request_text, "lxml")

# Find the first Summer Olympics table
tables = page.find_all("table", {"class": "wikitable"})
summer_olympics_table = tables[0]

# Extract rows from the table
rows = []
for row in summer_olympics_table.find_all("tr")[1:]:  # Skip header row
    columns = row.find_all(["th", "td"])
    if len(columns) > 1:  # Ensure enough columns exist
        city_link = columns[2].find("a")  # Find the link for the host city
        if city_link:
            city_name = city_link.text.strip()
            city_url = "https://fr.wikipedia.org" + city_link['href']
            rows.append((city_name, city_url))

def get_coordinates(city_url):
    try:
        city_page = BeautifulSoup(urlopen(city_url).read(), "lxml")
        # Find the span with id "coordinates"
        coordinates_span = city_page.find("span", {"id": "coordinates"})
        
        if coordinates_span:
            map_link = coordinates_span.find("a", {"class": "mw-kartographer-maplink"})
            if map_link:
                dms_coordinates = map_link.text.strip()
                
                # Splitting latitude and longitude
                lat_dms, lon_dms = dms_coordinates.split(", ")

                latitude = map_link.get("data-lat")
                longitude = map_link.get("data-lon")
                
                return lat_dms, lon_dms, latitude, longitude
            else:
                print("Coordinates link not found within the span.")
        else:
            print(f"Coordinates span not found for {city_url}.")
    except Exception as e:
        print(f"Error fetching {city_url}: {e}")
        return None, None
    return lat_dms, lon_dms, latitude, longitude

# Collecting city coordinates
city_data = []
for city_name, city_url in rows:
    lat_dms, lon_dms, latitude, longitude = get_coordinates(city_url)
    if lat_dms and lon_dms:
        city_data.append((city_name, lat_dms, lon_dms, latitude, longitude))
    time.sleep(1)

# Create DataFrame with city names and coordinates
data = pd.DataFrame(city_data, columns=["name", "latitude(DMS)", "longitude(DMS)", "latitude", "longitude"])

# Display the DataFrame
print(data)


              name    latitude((DMS)      longitude(DMS)         latitude  \
0          Athènes  37° 58′ 00″ nord     23° 43′ 00″ est       37.9666667   
1            Paris  48° 51′ 24″ nord      2° 21′ 07″ est        48.856578   
2      Saint-Louis  38° 38′ 53″ nord   90° 12′ 44″ ouest        38.648056   
3          Athènes  37° 58′ 00″ nord     23° 43′ 00″ est       37.9666667   
4          Londres  51° 30′ 26″ nord    0° 07′ 39″ ouest        51.507222   
5        Stockholm  59° 19′ 46″ nord     18° 04′ 07″ est        59.329444   
6           Berlin         52° 31′ N           13° 23′ E  52.516666666667   
7           Anvers      51° 13′ nord          4° 24′ est           51.217   
8            Paris  48° 51′ 24″ nord      2° 21′ 07″ est        48.856578   
9        Amsterdam      52° 22′ nord          4° 53′ est  52.366666666667   
10     Los Angeles      34° 03′ nord      118° 15′ ouest         34.05223   
11          Berlin         52° 31′ N           13° 23′ E  52.516666666667   

We then transform degree coordinates into numerical coordinates to build a map


In [86]:
#This part is not necessary as I got the decimal coordinates above. 
import re

def dms2dd(degrees, minutes, seconds, direction):
    dd = float(degrees) + float(minutes)/60 + float(seconds)/(60*60);
    if direction in ('S', 'O'):
        dd *= -1
    return dd

def parse_dms(dms):
    parts = re.split('[^\d\w]+', dms)
    #potentially needs an adaptation
    #[student]
    lat = dms2dd(parts[0], parts[1], parts[2], parts[3])
    #lng = dms2dd(parts[4], parts[5], parts[6], parts[7])
    return lat

#data['latitude'] = data['latitude'].apply(parse_dms)
#data['longitude'] = data['longitude'].apply(parse_dms)
print(data)

              name   latitude   longitude
0          Athènes  37.966667   23.716667
1            Paris  48.856667    2.351944
2      Saint-Louis  38.648056   90.212222
3          Athènes  37.966667   23.716667
4          Londres  51.507222    0.127500
5        Stockholm  59.329444   18.068611
6           Berlin        NaN         NaN
7           Anvers        NaN         NaN
8            Paris  48.856667    2.351944
9        Amsterdam        NaN         NaN
10     Los Angeles        NaN         NaN
11          Berlin        NaN         NaN
12           Tokyo  35.689444  139.691667
13         Londres  51.507222    0.127500
14         Londres  51.507222    0.127500
15        Helsinki  60.170833   24.937500
16       Melbourne  37.814167  144.968333
17            Rome  41.888611   12.486667
18           Tokyo  35.689444  139.691667
19          Mexico  19.419444   99.145556
20          Munich  48.150000   11.575000
21        Montréal  45.503333   73.586944
22          Moscou  55.752500   37

The map can be obtain with the following code

In [64]:
!pip install folium

In [69]:
import geopandas as gpd
from pathlib import Path
import folium
from IPython.display import display

gdf = gpd.GeoDataFrame(
    data, geometry=gpd.points_from_xy(data.longitude, data.latitude))

Path("leaflet").mkdir(parents=True, exist_ok=True)

center = gdf[['latitude', 'longitude']].mean().values.tolist()
sw = gdf[['latitude', 'longitude']].min().values.tolist()
ne = gdf[['latitude', 'longitude']].max().values.tolist()

m = folium.Map(location = center, tiles='openstreetmap')

# I can add marker one by one on the map
for i in range(0,len(gdf)):
    folium.Marker([gdf.iloc[i]['latitude'], gdf.iloc[i]['longitude']], popup=gdf.iloc[i]['name']).add_to(m)

m.fit_bounds([sw, ne])
display(m)

# Project

The objective of this project is to practice all concepts taught in the main lecture. Therefore, you will have to collect data around a thematic, identify a problematic, clean and format the data, provide some exploratory analysis and visualization, build models and evaluate them. You will have also to design dashboard and to storytell the whole pipeline.


The whole project will have two outputs describing the methodology and the results:
* A technical report targeting your datascientist colleagues
* An oral presentation targeting your CEO, chief, client. In this case, we assume that the audience is not specialized in data science. (But you also need to present the methodology to convince them to trust the results).


**Requests:**
* Team work of two people (same group throughout the semester)
* All your work should be stored on a git repo: [tutorial](https://github.com/baskiotisn/2IN013robot2023/blob/d979333fb80c9b6acd9515aaec040943d10d365c/docs/tutoriel_git.pdf)

**Remarks:**
There are also other libraries or issues you'll encounter when collecting data. Some of these are listed below, but don't be shy and interact with a search engine to solve your own issue!
* [Regular expressions](https://docs.python.org/3/howto/regex.html) might be useful!
* [API with authentication](https://www.geeksforgeeks.org/authentication-using-python-requests/)
*   [Selenium](https://selenium-python.readthedocs.io/) when pages are generated wia javascript scripts
* [Playwright](https://playwright.dev/) --> looks easier and more adapted than Selenium
* [Scrapy](https://scrapy.org/) for web crawling / or when you don't know the URL.[Tuto here](https://doc.scrapy.org/en/latest/intro/tutorial.html)
* [Summary of some difficulties in scrapping data from the web](https://www.zenrows.com/blog/web-scraping-challenges#page-structure-changes)

## Your daily task
Collecting data from the web and open data portals. Open data might include csv files, you can use them, but be aware that scraping should be your main acitivity in the dataset gathering.
You can begin to format the data, merge them to build dataframe that would be analyzed next week.

In [96]:
import requests
from bs4 import BeautifulSoup

# URL of the events page
url = "https://www.montpellier-france.com/whats-on/whats-on-this-week/"

# Headers to mimic a browser visit
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
}

# Request the webpage
response = requests.get(url, headers=headers)

# Check if the request was successful
if response.status_code == 200:
    # Parse the page content
    soup = BeautifulSoup(response.text, 'html.parser')

    # Find the events section (you may need to inspect the webpage to get the right tags/classes)
    events = soup.find_all('section', class_='infos')

    # Extract event details
    for event in events:
        title = event.find('div',class_="item-infos-title").get_text(strip=True) if event.find('div',class_="item-infos-title") else 'No title'
        date = event.find('span', class_='item-infos-date').get_text(strip=True) if event.find('span', class_='item-infos-date') else 'No date'
        location = event.find('adress').get_text(strip=True) if event.find('adress') else 'No location'

        print(f"Title: {title}")
        print(f"Date: {date}")
        print(f"Location: {location}")
        print("-" * 40)

else:
    print(f"Failed to fetch page, status code: {response.status_code}")


Title: JOURNEE FRANCO-ALLEMANDE PROJECTION-DÉBAT
Date: OnWednesday22Janat 19:00
Location: Montpellier
----------------------------------------
Title: NUIT DE LA LECTURE À ATRIUM
Date: OnThursday23Janat 19:30
Location: Montpellier
----------------------------------------
Title: ATELIER “VOYAGER AUTREMENT”
Date: OnThursday23Janat 19:00
Location: Castries
----------------------------------------
Title: EMANUEL GAT  FREEDOM SONATA
Date: From22until23January
Location: Montpellier
----------------------------------------
Title: CONCERT PETIT VINCE TRIO
Date: OnFriday24Janat 21:00
Location: Montpellier
----------------------------------------
Title: CONCERT EMILE LONDONIEN
Date: OnFriday24Jan
Location: Montpellier
----------------------------------------
Title: SOLIDARITÉ CAMBIER AU DRAPEAU ROUGE
Date: OnFriday24Janat 17:00
Location: Montpellier
----------------------------------------
Title: "IMPROVATAR", THÉÂTRE D'IMPROVISATION
Date: OnFriday24Janat 20:00
Location: Juvignac
----------------

In [97]:
import requests
from bs4 import BeautifulSoup

# URL of the events page
url = "https://lyon.citycrunch.fr/que-faire-a-lyon-ce-week-end-17-18-et-19-janvier-2025/2025/01/17/"

# Headers to mimic a browser request
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
}

# Sending GET request to fetch the page content
response = requests.get(url, headers=headers)

# Check if the request was successful
if response.status_code == 200:
    # Parse the page content with BeautifulSoup
    soup = BeautifulSoup(response.text, 'html.parser')

    # Find event sections (Adjust based on actual HTML structure)
    event_sections = soup.find_all('h3')

    # Extract event details
    for event in event_sections:
        title = event.get_text(strip=True)
        
        # Some events might have descriptions in the next <p> tag
        description = event.find_next('p').get_text(strip=True) if event.find_next('p') else 'No description available'

        print(f"Title: {title}")
        print(f"Description: {description}")
        print("-" * 40)

else:
    print(f"Failed to fetch page, status code: {response.status_code}")



Title: 09h00 >Grande vente de planteschez In-Sted (Lyon 3)
Description: 
----------------------------------------
Title: 20h00 >Les Tétons Fripons – cabaret burlesqueau Lavoir Public (Lyon 1)
Description: 
----------------------------------------
Title: 22h00 >« Save The Sonic »,  Boum de soutien au Sonic (Lyon 5)
Description: 
----------------------------------------
Title: 23h30 >This is Discoau Petit Salon (Lyon 7)
Description: 
----------------------------------------
Title: 10h00 >La Big Braderie du Collectif les Mainsà La Fabuleuse Cantine (Lyon 7)
Description: 
----------------------------------------
Title: 12h00 >Japan Feverà la Commune (Lyon 7)
Description: 
----------------------------------------
Title: 18h00 >Doudoune Partyau Gourmet Bar Confluence (Lyon 2)
Description: 
----------------------------------------
Title: 18h00 >Lugd’O Night,course d’orientation audépart à la Mulatière
Description: 
----------------------------------------
Title: 20h00 >Ciné Blind-testà l’Aqua

In [99]:
import requests
from bs4 import BeautifulSoup

# URL of the Paris Update events page
url = "https://www.parisupdate.com/"

# Headers to mimic a browser request
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
}

# Sending GET request to fetch the page content
response = requests.get(url, headers=headers)

# Check if the request was successful
if response.status_code == 200:
    # Parse the page content with BeautifulSoup
    soup = BeautifulSoup(response.text, 'html.parser')

    # Find the section containing events
    events_section = soup.find('div', class_='events-list')

    # Check if the events section exists
    if events_section:
        # Find all event items within the section
        event_items = events_section.find_all('div', class_='event-item')

        # Extract and print event details
        for event in event_items:
            title = event.find('h3').get_text(strip=True) if event.find('h3') else 'No title available'
            date = event.find('span', class_='event-date').get_text(strip=True) if event.find('span', class_='event-date') else 'No date available'
            description = event.find('p', class_='event-description').get_text(strip=True) if event.find('p', class_='event-description') else 'No description available'

            print(f"Title: {title}")
            print(f"Date: {date}")
            print(f"Description: {description}")
            print("-" * 40)
    else:
        print("Events section not found on the page.")
else:
    print(f"Failed to fetch page, status code: {response.status_code}")


Events section not found on the page.
